# VTE Timepoints, remove analysis after VTE occurred.

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time

### Teg-Hip
load_dotenv()
api_url = os.getenv("REDCAP_URL")
api_key_hip = os.getenv("REDCAP_TEG_HIP_API_KEY")

p1 = RedcapProcessor(api_url, api_key_hip)
df_hip_original = p1.fetch_and_process()

df_hip_demo=p1.get_all_demographics()
df_hip_lab = p1.get_all_labs()
df_hip = p1.get_full_dataframe()


KeyboardInterrupt: 

In [ ]:
# p1.get_demo('TH-001')

In [ ]:
# p1.get_draws('TH-001')

In [ ]:
# df_hip_original[['StudyID', 'very_fit','well','managing_well','vulnerable','mildly_frail','moderately_frail','severely_frail','very_severely_frail','terminally_ill', 'MA']].head(40)

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time

# ### Hip Pathway
load_dotenv()
api_url = os.getenv("REDCAP_URL")
api_key_pathway = os.getenv("REDCAP_HIP_PATHWAY_API_KEY")

p2 = RedcapProcessor(api_url, api_key_pathway)


df_pathway_original = p2.fetch_and_process()
df_pathway_demo=p2.get_all_demographics()
df_pathway_lab = p2.get_all_labs()
# df_pathway = p2.get_full_dataframe()

Data Fetched
Records Successfully Built!
********* This study has 33 patients ************


,draw_group,Time,R_time,K_time,Alpha_Angle,MA,LY30,ACT,ADP-agg,ADP-inh,ADP-ma,AA-agg,AA-inh,AA-ma
26,Post-Operative,POD3,0.4,0.8,79.4,68.3,0.0,87.9,73.1,26.9,56.8,19.2,80.8,33.0
27,Post-Operative,POD1,0.7,1.1,76.0,64.2,0.0,116.0,59.4,40.6,45.3,2.4,97.6,19.2
33,2wk follow-up (±3 days),Week2,0.3,0.8,78.3,68.0,0.0,78.5,88.8,11.2,63.0,94.4,5.6,65.8
35,3M follow-up (±14 days),Month3,0.5,1.3,74.9,61.7,0.0,97.3,82.6,17.4,53.1,90.0,10.0,56.8
36,6wk follow-up (±7 days),Week6,0.5,1.1,75.0,63.1,0.0,97.3,85.4,14.6,54.8,93.7,6.3,58.9
37,4wk follow-up (±7 days),Week4,0.4,0.9,77.2,66.0,0.0,87.9,85.4,14.6,58.3,91.3,8.7,61.3
40,Admission,Admission,0.4,1.2,75.8,62.6,0.0,87.9,56.7,43.3,41.9,94.2,5.8,59.4


,draw_group,Time,Hemoglobin,Creatinine
26,Post-Operative,POD3,117,64
27,Post-Operative,POD1,121,57
32,Admission,Admission,139,58
33,2wk follow-up (±3 days),Week2,NaN,NaN
35,3M follow-up (±14 days),Month3,NaN,NaN
36,6wk follow-up (±7 days),Week6,NaN,NaN
37,4wk follow-up (±7 days),Week4,NaN,NaN
40,Admission,Admission,NaN,NaN


,StudyID,draw_group,Time,R_time,K_time,Alpha_Angle,MA,LY30,ACT,ADP-agg,ADP-inh,ADP-ma,AA-agg,AA-inh,AA-ma,Hemoglobin,Creatinine
8,HPA-004,2wk follow-up (±3 days),Week2,0.3,0.8,78.3,68.0,0.0,78.5,88.8,11.2,63.0,94.4,5.6,65.8,NaN,NaN
9,HPA-004,3M follow-up (±14 days),Month3,0.5,1.3,74.9,61.7,0.0,97.3,82.6,17.4,53.1,90.0,10.0,56.8,NaN,NaN
10,HPA-004,4wk follow-up (±7 days),Week4,0.4,0.9,77.2,66.0,0.0,87.9,85.4,14.6,58.3,91.3,8.7,61.3,NaN,NaN
11,HPA-004,6wk follow-up (±7 days),Week6,0.5,1.1,75.0,63.1,0.0,97.3,85.4,14.6,54.8,93.7,6.3,58.9,NaN,NaN
12,HPA-004,Admission,Admission,0.4,1.2,75.8,62.6,0.0,87.9,56.7,43.3,41.9,94.2,5.8,59.4,139,58
13,HPA-004,Post-Operative,POD1,0.7,1.1,76.0,64.2,0.0,116.0,59.4,40.6,45.3,2.4,97.6,19.2,121,57
14,HPA-004,Post-Operative,POD3,0.4,0.8,79.4,68.3,0.0,87.9,73.1,26.9,56.8,19.2,80.8,33.0,117,64


In [3]:
import pandas as pd
import numpy as np

# Sample data
data = {
    'StudyID': ['HPA-004']*8,
    'draw_group': ['2wk follow-up (±3 days)', '3M follow-up (±14 days)', '4wk follow-up (±7 days)',
                   '6wk follow-up (±7 days)', 'Admission', 'Admission', 'Post-Operative', 'Post-Operative'],
    'Time': ['Week2','Month3','Week4','Week6','Admission', np.nan,'POD1','POD3'],
    'R_time':[0.3,0.5,0.4,0.5,0.4, np.nan,0.7,0.4],
    'K_time':[0.8,1.3,0.9,1.1,1.2,np.nan,1.1,0.8],
    'Alpha_Angle':[78.3,74.9,77.2,75.0,75.8,np.nan,76.0,79.4],
    'MA':[68.0,61.7,66.0,63.1,62.6,np.nan,64.2,68.3],
    'LY30':[0.0,0.0,0.0,0.0,0.0,np.nan,0.0,0.0],
    'ACT':[78.5,97.3,87.9,97.3,87.9,np.nan,116.0,87.9],
    'ADP-agg':[88.8,82.6,85.4,85.4,56.7,np.nan,59.4,73.1],
    'ADP-inh':[11.2,17.4,14.6,14.6,43.3,np.nan,40.6,26.9],
    'ADP-ma':[63.0,53.1,58.3,54.8,41.9,np.nan,45.3,56.8],
    'AA-agg':[94.4,90.0,91.3,93.7,94.2,np.nan,2.4,19.2],
    'AA-inh':[5.6,10.0,8.7,6.3,5.8,np.nan,97.6,80.8],
    'AA-ma':[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,19.2,33.0],
    'Hemoglobin':[np.nan,np.nan,np.nan,np.nan,np.nan,139,121,117],
    'Creatinine':[np.nan,np.nan,np.nan,np.nan,np.nan,58,57,64]
}

df = pd.DataFrame(data)

# Combine rows with the same draw_group by keeping non-null values
combined = df.groupby('draw_group', as_index=False).agg(lambda x: x.dropna().iloc[0] if len(x.dropna()) > 0 else np.nan)

display(combined)


,draw_group,StudyID,Time,R_time,K_time,Alpha_Angle,MA,LY30,ACT,ADP-agg,ADP-inh,ADP-ma,AA-agg,AA-inh,AA-ma,Hemoglobin,Creatinine
0,2wk follow-up (±3 days),HPA-004,Week2,0.3,0.8,78.3,68.0,0.0,78.5,88.8,11.2,63.0,94.4,5.6,NaN,NaN,NaN
1,3M follow-up (±14 days),HPA-004,Month3,0.5,1.3,74.9,61.7,0.0,97.3,82.6,17.4,53.1,90.0,10.0,NaN,NaN,NaN
2,4wk follow-up (±7 days),HPA-004,Week4,0.4,0.9,77.2,66.0,0.0,87.9,85.4,14.6,58.3,91.3,8.7,NaN,NaN,NaN
3,6wk follow-up (±7 days),HPA-004,Week6,0.5,1.1,75.0,63.1,0.0,97.3,85.4,14.6,54.8,93.7,6.3,NaN,NaN,NaN
4,Admission,HPA-004,Admission,0.4,1.2,75.8,62.6,0.0,87.9,56.7,43.3,41.9,94.2,5.8,NaN,139.0,58.0
5,Post-Operative,HPA-004,POD1,0.7,1.1,76.0,64.2,0.0,116.0,59.4,40.6,45.3,2.4,97.6,19.2,121.0,57.0


In [ ]:
df_pathway_original[df_pathway_original['StudyID']=='HPA-004'][['StudyID','redcap_event_name','Time','R_time','K_time','Alpha_Angle','MA','LY30','ACT',
            'ADP-agg','ADP-inh','ADP-ma','AA-agg','AA-inh','AA-ma',
            'Draw_date_teg','Hemoglobin', 'Creatinine', 'Draw_date_lab'
        ]]

,StudyID,redcap_event_name,Time,R_time,K_time,Alpha_Angle,MA,LY30,ACT,ADP-agg,ADP-inh,ADP-ma,AA-agg,AA-inh,AA-ma,Draw_date_teg,Hemoglobin,Creatinine,Draw_date_lab
25,HPA-004,Post-Operative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
26,HPA-004,Post-Operative,POD3,0.4,0.8,79.4,68.3,0.0,87.9,73.1,26.9,56.8,19.2,80.8,33.0,2024-02-23 07:00:00,117,64,2024-02-23 02:21:00
27,HPA-004,Post-Operative,POD1,0.7,1.1,76.0,64.2,0.0,116.0,59.4,40.6,45.3,2.4,97.6,19.2,2024-02-21 10:30:00,121,57,2024-02-21 02:26:00
28,HPA-004,Post-Operative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
29,HPA-004,Post-Operative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
30,HPA-004,Post-Operative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
31,HPA-004,Post-Operative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
32,HPA-004,Admission/Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-19 20:08:00,139,58,2024-02-19 20:08:00
33,HPA-004,2wk follow-up (±3 days),Week2,0.3,0.8,78.3,68.0,0.0,78.5,88.8,11.2,63.0,94.4,5.6,65.8,NaT,NaN,NaN,NaT
34,HPA-004,Admission/Baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT


In [ ]:
df_pathway_lab[df_pathway_lab['StudyID']=='HPA-004'][['StudyID','draw_group','Hemoglobin', 'MA']]

,StudyID,draw_group,Hemoglobin,MA
1,HPA-004,2wk follow-up (±3 days),NaN,68.0
2,HPA-004,3M follow-up (±14 days),NaN,61.7
3,HPA-004,4wk follow-up (±7 days),NaN,66.0
4,HPA-004,6wk follow-up (±7 days),NaN,63.1
62,HPA-004,Post-Operative,121,64.2
63,HPA-004,Admission,139,NaN


The problem is that I need the "time_injury_lab_hours" to be from where there was no Time (from the row before we fill it with "Admission")

## I should have a TEG data and another for LAB data

In [ ]:
df_pathway_lab.to_excel('TEST___df_pathway_lab____________MERG.xlsx', index=False)

In [ ]:
# p2.get_demo('HPA-028')

In [ ]:
p2.get_draws('HPA-028')

In [ ]:
df_pathway[['Injury_date','Draw_date_teg','Draw_date_lab','Surgery_date','time_injury_to_surgery_hours']]

In [ ]:
p2.get_draws('HPA-028')

In [ ]:
df_pathway[df_pathway['StudyID']=='HPA-028'][['Hemoglobin','Draw_date_lab','Time','time_injury_lab_hours','time_injury_teg_hours']]

In [ ]:
df_pathway[~df_pathway['time_injury_lab_hours'].isnull()&df_pathway['Time'].isnull()][['time_injury_lab_hours','time_injury_teg_hours']]

In [ ]:
p2.get_draws('HPA-028').dropna(subset=[ 'Time','Draw_id','MA','AA-ma','Creatinine'], how='all')

In [ ]:
load_dotenv()

### Teg-Femur
api_url = os.getenv("REDCAP_URL")
api_key_femur = os.getenv("REDCAP_TEG_FEMUR_API_KEY")

p3 = RedcapProcessor(api_url, api_key_femur)



df_femur_original = p3.fetch_and_process()
df_femur_demo=p3.get_all_demographics()
df_femur_lab = p3.get_all_labs()
df_femur = p3.get_full_dataframe()

In [ ]:
df_femur_demo[df_femur_demo['VTE']=="Yes"][['StudyID','VTE','DVT','PE','VTE_type','Treatment','Pre_op_med']].reset_index(drop=True)

In [ ]:
load_dotenv()

### Teg-Pelvis
api_url = os.getenv("REDCAP_URL")
api_key_pelvis = os.getenv("REDCAP_TEG_PELVIS_API_KEY")

p4 = RedcapProcessor(api_url, api_key_pelvis)

df_pelvis_original = p4.fetch_and_process()
df_pelvis_demo=p4.get_all_demographics()
df_pelvis_lab = p4.get_all_labs()
df_pelvis = p4.get_full_dataframe()

In [ ]:
df_pelvis_demo[df_pelvis_demo['VTE']=="Yes"][['StudyID','VTE','DVT','PE','VTE_type','Withdrawn','Death','Pre_op_med']].reset_index(drop=True)

In [ ]:
df_pelvis_original.head(20)[['StudyID','Draw_date_lab','Draw_date_teg', "Hemoglobin",'MA','Time','Injury_date','Pre_op_med']]

In [ ]:
df_pelvis_demo.head()[['StudyID','Surgery_date']]

In [ ]:
ALL_TEG_DEMO=pd.concat([df_hip_demo,df_pathway_demo,df_femur_demo,df_pelvis_demo])
ALL_TEG_DEMO.to_excel('ALL_TEG_DEMO.xlsx', index=False)

In [ ]:
ALL_TEG_BLOOD=pd.concat([all_blood_hip,all_blood_pathway,all_blood_femur,all_blood_pelvis])
ALL_TEG_BLOOD.to_excel('ALL_TEG_BLOOD.xlsx', index=False)

In [ ]:
ALL_TEG_BLOOD['VTE']

In [ ]:
ALL_TEG=pd.merge(ALL_TEG_DEMO,ALL_TEG_BLOOD.drop(columns=['Injury_date','Surgery_date','Pre_op_med','Study','VTE','VTE_type','PE','DVT']), on=['StudyID'], how='outer', indicator=True).drop_duplicates(keep='first')
ALL_TEG.to_excel('ALL_TEG.xlsx', index=False)

In [ ]:
ALL_TEG.columns

In [ ]:
display_summary_tables(ALL_TEG, filter_col='VTE', extra_col='VTE_type')


In [ ]:
display_summary_tables(ALL_TEG, filter_col='Death')

In [ ]:
display_summary_tables(ALL_TEG, filter_col='Withdrawn')

In [ ]:
display_value_counts_per_study(ALL_TEG_DEMO, 'VTE')

In [ ]:
display_value_counts_per_study(ALL_TEG_DEMO, 'Pre_op_med')

In [ ]:
display_value_counts_per_study(ALL_TEG_DEMO, 'Sex')

In [ ]:
display_value_counts_per_study(ALL_TEG_DEMO, 'AO_OTA')

In [ ]:
display_summary_tables(ALL_TEG, filter_col='blood_rbc_yn')

In [ ]:
display_value_counts_per_study(ALL_TEG_DEMO, 'blood_rbc_yn')

In [ ]:
display(ALL_TEG[ALL_TEG['injury_to_lab_hrs']<0][['StudyID','Time','Injury_date','Draw_date_lab','Draw_date_teg','injury_to_lab_hrs','injury_to_teg_hrs','surgery_to_lab_hrs','surgery_to_teg_hrs']])

# Duploicate ERRORS

In [ ]:
ALL_TEG[ALL_TEG.duplicated(subset=['StudyID','Time'])][['StudyID','Time','MA']]

In [ ]:
df_demo=pd.concat([df_hip_demo,df_pathway_demo]).reset_index(drop=True)


df_demo_oac = df_demo[df_demo['Pre_op_med'].isin(['DOAC','Warfarin']) ]
df_blood_draws = pd.concat([p1.get_all_blood_draws(), p2.get_all_blood_draws()]).drop_duplicates().reset_index(drop=True)

# ONLY DOAC - Hemoglobin drop

In [ ]:
df_blood_draws_oac = df_blood_draws[df_blood_draws['StudyID'].isin(df_demo_oac['StudyID'])]

In [ ]:
df_blood_draws_oac

In [ ]:
analyze_hgb(df_blood_draws_oac, pod_time="POD1")

In [ ]:
analyze_hgb(df_blood_draws_oac, pod_time="POD3")

In [ ]:
# df_blood_draws[df_blood_draws['Time']=='Week2'].sort_values('StudyID')

In [ ]:
df_blood_draws.head()

In [ ]:
df_blood_draws[df_blood_draws['Time'].isin(['Admission','Pre-Op','PFD1','Pre-Op','PFD2','4 hr Pre-OP','Pre-draw','PFD3','PFD4'])].head(10)

## Created two columns to pick depending on if we want pre-op closest to injury or if we want pre-op closest to surgery

In [ ]:
preop_timepoints = ['Admission','Pre-Op','PFD1','Pre-Op','PFD2','4 hr Pre-OP','Pre-draw','PFD3','PFD4']

# Initialize new columns
df_blood_draws['closest_to_injury'] = 'No'
df_blood_draws['closest_to_surgery'] = 'No'


columns_to_check = ['R_time', 'K_time', 'MA', 'LY30', 'ACT','Alpha_Angle']

for patient_id, group in df_blood_draws.groupby('StudyID'):
    preop_draws = group[group['Time'].isin(preop_timepoints)].copy()
    
    if not preop_draws.empty:
        # Filter out draws where all columns are NaN
        valid_draws = preop_draws.dropna(subset=columns_to_check, how='all')
        
        if not valid_draws.empty:
            # Closest to injury
            closest_idx = valid_draws['injury_to_lab_hrs'].idxmin()
            df_blood_draws.loc[closest_idx, 'closest_to_injury'] = 'Yes'
            
            # Closest to surgery
            farthest_idx = valid_draws['injury_to_lab_hrs'].idxmax()
            df_blood_draws.loc[farthest_idx, 'closest_to_surgery'] = 'Yes'
        else:
            # If all draws are invalid, fallback to first/last pre-op draw
            df_blood_draws.loc[preop_draws.index[0], 'closest_to_injury'] = 'Yes'
            df_blood_draws.loc[preop_draws.index[-1], 'closest_to_surgery'] = 'Yes'
            
# # Loop over each patient
# for patient_id, group in df_blood_draws.groupby('StudyID'):
#     preop_draws = group[group['Time'].isin(preop_timepoints)].copy()
    
#     if not preop_draws.empty:
#         # Closest to injury
#         if preop_draws['time_from_injury_to_draw_hours'].notna().any():
#             closest_idx = preop_draws['time_from_injury_to_draw_hours'].idxmin()
#         elif (preop_draws['Time'] == 'Admission').any():
#             closest_idx = preop_draws[preop_draws['Time'] == 'Admission'].index[0]
#         else:
#             # Fallback: just take the first pre-op draw
#             closest_idx = preop_draws.index[0]
#         df_blood_draws.loc[closest_idx, 'closest_to_injury'] = 'Yes'
        
#         # Closest to surgery (farthest from injury)
#         if preop_draws['time_from_injury_to_draw_hours'].notna().any():
#             farthest_idx = preop_draws['time_from_injury_to_draw_hours'].idxmax()
#         elif (preop_draws['Time'] == 'Admission').any():
#             farthest_idx = preop_draws[preop_draws['Time'] == 'Admission'].index[0]
#         else:
#             # Fallback: just take the last pre-op draw
#             farthest_idx = preop_draws.index[-1]
#         df_blood_draws.loc[farthest_idx, 'closest_to_surgery'] = 'Yes'

# Update 'Time' for closest to injury
df_blood_draws['Time'] = np.where(
    df_blood_draws['closest_to_injury'] == 'Yes',
    'Pre_op',
    df_blood_draws['Time'])

In [ ]:
# df_blood_draws[df_blood_draws['StudyID']=='TH-137']

In [ ]:
df_blood_draws[df_blood_draws['closest_to_injury']=='Yes'].head()

In [ ]:
columns = ['Sex', 'Age', 'BMI','VTE_type','VTE']
groupby = 'Pre_op_med'
categorical = ['Sex','VTE_type','VTE']

nonnormal=['BMI'] 
display(TableOne(df_demo, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby))

In [ ]:
columns = ['R_time', 'K_time',  'MA', 'LY30', 'ACT','Alpha_Angle'] 
groupby = 'Pre_op_med'
categorical = []


for timepoint in ['Pre_op','POD1', 'POD3', 'POD5', 'POD7', 'Week2', 'Week4', 'Week6', 'Month3']:
    print(timepoint)
    print('++++++++++++++++++++++++++++++++++++++++++++')
    df=df_blood_draws[df_blood_draws['Time']==timepoint]


    

    nonnormal=[ 'R_time', 'K_time',  'MA', 'LY30', 'ACT','Alpha_Angle'] 
    display(TableOne(df, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby))

In [ ]:
df_blood_draws.columns

In [ ]:
for col in columns:
    df_blood_draws[col] = pd.to_numeric(df_blood_draws[col], errors='coerce')

    
plot_variables_over_time(df_blood_draws,
                         hue='VTE', 
    style='VTE',
    palette={'Yes':"#ff3636", 'No':"#030303"},
    # var_labels=var_labels,
    xlabel="Timepoints",
)

In [ ]:
df_blood_draws[(df_blood_draws['StudyID']=='TH-190')&(df_blood_draws['Time']=='Pre_op')][['Time','LY30','Pre_op_med']]

In [ ]:
display(all_blood_hip[all_blood_hip['StudyID']=='TH-188'][['Time','LY30']])